In [ ]:
from pathlib import Path

from utils.dataset_utils import get_politicalness_datasets_for_leave_one_in_benchmark
from utils.model_utils import evaluate_models, get_custom_politicalness_models

In [ ]:
GET_DATASETS = lambda: get_politicalness_datasets_for_leave_one_in_benchmark()
DATASET_SAMPLE_SIZE = 1_000

datasets = list(
    map(
        lambda dataset: dataset
        .take_even_class_distribution_sample(DATASET_SAMPLE_SIZE)
        .transform_for_inference()
        .to_huggingface(),
        GET_DATASETS(),
    )
)

In [ ]:
# Lambda, so that the generator can be reused.
GET_MODELS = lambda: get_custom_politicalness_models(Path("dataset_benchmark") / "leave_one_in" / "politicalness")
TRUNCATE_TOKENS = True

evaluate_models(
    GET_MODELS,
    datasets,
    TRUNCATE_TOKENS
)